In [ ]:
import pandas as pd
import sqlalchemy as sql
from sql_functions import get_sql_config
from sql_functions import get_engine
from sql_functions import push_to_database
from update import update_sql_table
import psycopg2

csv datei einlesen

In [ ]:
sachsen_22 = pd.read_csv("data/sachsen_22.csv", delimiter= ";", encoding= 'windows-1252',dtype={'Titel':str})
sachsen_22.head(20)
sachsen_22.info()

In [ ]:
sachsen_22.head(20)

columns struktur anpassen

In [ ]:
sachsen_22.columns = sachsen_22.columns.str.replace(" ", "_")
sachsen_22.columns = sachsen_22.columns.str.lower()
sachsen_22.head()

In [ ]:
sachsen_22.rename(columns = {'einzelplan':'ep' }, inplace = True)
sachsen_22.head()

Columns die nicht gebraucht werden droppen

In [ ]:
sachsen_22.drop('ansatz_2021_in_t€', inplace=True, axis=1)


In [ ]:
sachsen_22.drop('tg', inplace=True, axis=1)

In [ ]:
sachsen_22.head()

Columns umbenennen

In [ ]:
sachsen_22.rename(columns={'ansatz_2022_in_t€':'ansatz_2022_in_euro'}, inplace=True)

In [ ]:
sachsen_22.tail()

erneut ein column gedropped

In [ ]:
sachsen_22.drop('bezeichnung.2', inplace=True, axis=1)


In [ ]:
sachsen_22.head()

NaN Werte in 0 umgewandelt

In [ ]:
sachsen_22["ansatz_2022_in_euro"] = sachsen_22["ansatz_2022_in_euro"].fillna('0,0')

In [ ]:
sachsen_22.head()

Ep (einzeplan) angepasst und vereinheitlicht

In [ ]:
sachsen_22['ep'] = sachsen_22['ep'].astype(str)
sachsen_22['ep'] = sachsen_22['ep'].apply(lambda x:x.zfill(2))

In [ ]:
sachsen_22.head()

State-ID hinzugefügt

In [ ]:
sachsen_22.insert(loc=10,
          column='state_id',
          value=13)
sachsen_22.insert(loc=11,
            column='year',
            value=2022)

In [ ]:
sachsen_22.head()

Struktur ders Columns strukturiert und umgewandelt in einem float und *1000 multipliziert

In [ ]:
sachsen_22.ansatz_2022_in_euro = sachsen_22['ansatz_2022_in_euro'].str.replace('.', '').str.replace(",", ".").astype(float)
sachsen_22.ansatz_2022_in_euro = sachsen_22['ansatz_2022_in_euro'].apply(lambda x:int(x*1000))

In [ ]:
sachsen_22.head()

In [ ]:
sachsen_22.info()

Columns umgewandelt und strukturiert 

In [ ]:
sachsen_22.titel=sachsen_22.titel.astype(str)

In [ ]:
sachsen_22['gruppe']=sachsen_22.titel.str[:3]

In [ ]:
sachsen_22['counter']=sachsen_22.titel.str[3:]

In [ ]:
sachsen_22['ansatz_2022_in_euro'] =sachsen_22[['ansatz_2022_in_euro', 'gruppe']].apply(lambda x:-x[0] if int(x[1]) > 400 else x[0], axis=1)

In [ ]:
sachsen_22.head()

columns einnahme/ausgabe und titel gedropped

In [ ]:
sachsen_22.drop('einnahme/ausgabe', inplace=True, axis=1)
sachsen_22.drop('titel', inplace=True, axis=1)

In [ ]:
sachsen_22.head()

Kapitel neu strukturiert und verinheitlicht

In [ ]:
sachsen_22['kapitel'] = sachsen_22['kapitel'].astype(str)
sachsen_22['kapitel'] = sachsen_22['kapitel'].str[-2:]

In [ ]:
sachsen_22.tail()

In [ ]:
sachsen_22['kapitel'] = sachsen_22['kapitel'].apply(lambda x:x.zfill(3))
sachsen_22['fkz'] = sachsen_22['fkz'].astype(str)
sachsen_22['fkz'] = sachsen_22['fkz'].apply(lambda x:x.zfill(3))

In [ ]:
sachsen_22.head()


Neue DF erstellen

In [ ]:
fkz_index_df = sachsen_22[['fkz','bezeichnung_fkz']].copy()

In [ ]:
fkz_index_df.info()

In [ ]:

#fkz_index_df[fkz_index_df.duplicated(['fkz', 'bezeichnung_fkz'], keep=False)]
fkz_index_df.drop_duplicates(inplace=True)

In [ ]:
fkz_index_df.info()

In [ ]:
sachsen_22

In [ ]:
sachsen_ep = sachsen_22[['ep','bezeichnung', 'state_id', 'year']].copy()
sachsen_ep.rename(columns={'bezeichnung':'ep_txt'}, inplace=True)
sachsen_ep.drop_duplicates(inplace=True)

sachsen_kapitel = sachsen_22[['ep','kapitel', 'bezeichnung.1', 'state_id', 'year']].copy()
sachsen_kapitel.rename(columns={'bezeichnung.1':'kapitel_txt'}, inplace=True)
sachsen_kapitel.drop_duplicates(inplace=True)

sachsen_22 = sachsen_22.drop(['bezeichnung', 'bezeichnung.1', 'bezeichnung_fkz'], axis=1)
sachsen_22.rename(columns={'ansatz_2022_in_euro':'amount'}, inplace=True)

sachsen_22

In [ ]:
sachsen_ep

push it to the limit

In [ ]:
get_sql_config()
engine = get_engine()

In [ ]:
table_name = 'einzelplaene'
update_sql_table(sachsen_ep,table_name)
#push_to_database(sachsen_ep, "einzelplaene", engine, "capstone_public_budgeting")


In [ ]:
table_name = 'kapitel'
update_sql_table(sachsen_kapitel,table_name)
#push_to_database(sachsen_kapitel, "kapitel", engine, "capstone_public_budgeting")

In [ ]:
#push_to_database(sachsen_22, "budget_all", engine, "capstone_public_budgeting")